In [1]:
import pyBigWig
import h5py
import numpy as np
import matplotlib.pyplot as plt
from pyliftover import LiftOver
from tqdm import tqdm

In [2]:
sequences_bed_path = '/clusterfs/nilah/oberon/datasets/basenji2_human_training/sequences.bed'
records_prefix = '/clusterfs/nilah/oberon/datasets/basenji/cage/tfrs_seq-targets_basenji_test-train-valid_CAGE_'
shah_prefix = '/clusterfs/nilah/oberon/datasets/lamina_association_shah/'
hek_dimelo_prefix = '/clusterfs/nilah/oberon/datasets/cs282a/hek293/'
gm_dimelo_prefix = '/clusterfs/nilah/oberon/datasets/cs282a/gm12878/'
shah_tracks = ['GSM5669183_CardiacMyocytes_rep1_LB1.bigwig',
               'GSM5669188_APS_rep1_LB1.bigwig',
               'GSM5669193_BorderEctoderm_rep1_LB1.bigwig',
               'GSM5669198_D5Midbrain_rep1_LB1.bigwig',
               'GSM5669203_DefEctoderm_rep1_LB1.bigwig',
               'GSM5669209_EarlySomite_rep1_LB1.bigwig',
               'GSM5669214_EndoProgenitor_rep1_LB1.bigwig',
               'GSM5669220_Epicardium_rep1_LB1.bigwig',
               'GSM5669226_H9ESC_rep1_LB1.bigwig',
               'GSM5669232_Liver_rep1_LB1.bigwig',
               'GSM5669238_MidHindgut_rep1_LB1.bigwig',
               'GSM5669244_ParaxMesoderm_rep1_LB1.bigwig']

dataset_filepath = '/clusterfs/nilah/oberon/datasets/cs282a/dataset_14-lmnb1_4-cpg.h5'

In [3]:
hg38_to_t2tv1 = LiftOver('/clusterfs/nilah/oberon/genomes/hg38.t2t-chm13-v1.0.over.chain.gz')
t2tv1_to_t2tv1_1 = LiftOver('/clusterfs/nilah/oberon/genomes/v1.0_to_v1.1_rdna_merged.chain')

In [4]:
###########################################################
# Test how many of our sequences will fail to map
###########################################################
with open(sequences_bed_path,'r') as bed_file:
    failed = 0
    for index,bed_line in enumerate(bed_file):
        location_fields = bed_line.split('\t')
        chrom = str(location_fields[0])
        start = int(location_fields[1])+8192
        end = int(location_fields[2])
        t2t1_coordinate = hg38_to_t2tv1.convert_coordinate(chrom,start)
        if len(t2t1_coordinate)>0:
            t2t1_1_coordinate = t2tv1_to_t2tv1_1.convert_coordinate(t2t1_coordinate[0][0],t2t1_coordinate[0][1])
            if len(t2t1_1_coordinate)==0:
                failed+=1
        else:
            failed+=1
        
print(failed)

60


In [7]:
#######################################################
#
#######################################################
old_settings = np.seterr(invalid='ignore')
buffer_at_ends = 8192
seqbedlen = 38171
with open(sequences_bed_path,'r') as bed_file, h5py.File(dataset_filepath,'w') as f:
    
    f.create_dataset(
        '128bp_bins',
        shape=(seqbedlen,896,18),
        chunks=(1,896,18),
        compression='gzip',
        compression_opts=9,
    )
    f.create_dataset(
        'single_bin',
        shape=(seqbedlen,1,18),
        chunks=(1,1,18),
        compression='gzip',
        compression_opts=9,
    )
    
    for index,bed_line in tqdm(enumerate(bed_file),total=38171):
        location_fields = bed_line.split('\t')
        chrom = str(location_fields[0])
        start = int(location_fields[1])+buffer_at_ends
        end = int(location_fields[2])-buffer_at_ends
        shah_896 = np.zeros([896,12])
        shah_1 = np.zeros([1,12])
        for track_idx,shah_track in enumerate(shah_tracks):
            with pyBigWig.open(shah_prefix+shah_track) as bw:
                contigs = bw.chroms()
                contig_length = contigs[chrom]
                values = bw.values(chrom,start,end)
                # The track values stored were, per the paper, the log2 of ChIP enrichment
                shah_values = np.power(2,np.nan_to_num(np.array(values),-2))
                # We end up already on the appropriate 0-500 scale for the 128bp bins
                shah_896[:,track_idx] = np.sum(shah_values.reshape(-1,128),axis=1)
                # Summing across all bins we need to normalize back down
                shah_1[:,track_idx] = np.sum(shah_values)/896
        f['128bp_bins'][index,:,0:12] = shah_896
        f['single_bin'][index,:,0:12] = shah_1
        t2t1_coordinate = hg38_to_t2tv1.convert_coordinate(chrom,start)
        if len(t2t1_coordinate)>0:
            t2t1_1_coordinate = t2tv1_to_t2tv1_1.convert_coordinate(t2t1_coordinate[0][0],t2t1_coordinate[0][1])
            if len(t2t1_1_coordinate)>0:
                t2t_chrom = t2t1_1_coordinate[0][0]
                t2t_start = t2t1_1_coordinate[0][1]
                t2t_end = t2t_start + 114688
                with pyBigWig.open(f'{hek_dimelo_prefix}{chrom}/hek293_dimelo_feb2022_{chrom}_bigwig_modmA.bw') as bw:
                    contigs = bw.chroms()
                    contig = list(contigs.keys())[0]
                    contig_length = contigs[contig]
                    if t2t_end<contig_length:
                        clean_values = np.nan_to_num(np.array(bw.values(contig,t2t_start,t2t_end)))
                        dimelo_hek_modA_128 = np.sum(clean_values.reshape(-1,128),axis=1)
                        dimelo_hek_modA_single = np.sum(clean_values)
                    else:
                        dimelo_hek_modA_128 = np.zeros(896)
                        dimelo_hek_modA_single = np.nan
                with pyBigWig.open(f'{hek_dimelo_prefix}{chrom}/hek293_dimelo_feb2022_{chrom}_bigwig_validmA.bw') as bw:
                    contigs = bw.chroms()
                    contig = list(contigs.keys())[0]
                    contig_length = contigs[contig]
                    if t2t_end<contig_length:
                        clean_values = np.nan_to_num(np.array(bw.values(contig,t2t_start,t2t_end)))
                        dimelo_hek_validA_128 = np.sum(clean_values.reshape(-1,128),axis=1)
                        dimelo_hek_validA_single = np.sum(clean_values)
                    else:
                        dimelo_hek_validA_128 = np.zeros(896)
                        dimelo_hek_validA_single = np.nan                       
                    
                hek_mA_ratio_128 = np.nan_to_num(dimelo_hek_modA_128/dimelo_hek_validA_128)
                hek_mA_ratio_single = np.nan_to_num(dimelo_hek_modA_single/dimelo_hek_validA_single)
                # Scaling on these is 0 to 1, and we want 0 to 500
                f['128bp_bins'][index,:,12] = 500*hek_mA_ratio_128
                f['single_bin'][index,:,12] = 500*hek_mA_ratio_single
                
                with pyBigWig.open(f'{gm_dimelo_prefix}{chrom}/gm12878_dimelo_feb2022_{chrom}_bigwig_modmA.bw') as bw:
                    contigs = bw.chroms()
                    contig = list(contigs.keys())[0]
                    contig_length = contigs[contig]
                    if t2t_end<contig_length:
                        clean_values = np.nan_to_num(np.array(bw.values(contig,t2t_start,t2t_end)))
                        dimelo_gm_modA_128 = np.sum(clean_values.reshape(-1,128),axis=1)
                        dimelo_gm_modA_single = np.sum(clean_values)
                    else:
                        dimelo_gm_modA_128 = np.zeros(896)
                        dimelo_gm_modA_single = np.nan                      
                with pyBigWig.open(f'{gm_dimelo_prefix}{chrom}/gm12878_dimelo_feb2022_{chrom}_bigwig_validmA.bw') as bw:
                    contigs = bw.chroms()
                    contig = list(contigs.keys())[0]
                    contig_length = contigs[contig]
                    if t2t_end<contig_length:
                        clean_values = np.nan_to_num(np.array(bw.values(contig,t2t_start,t2t_end)))
                        dimelo_gm_validA_128 = np.sum(clean_values.reshape(-1,128),axis=1)
                        dimelo_gm_validA_single = np.sum(clean_values)
                    else:                      
                        dimelo_gm_validA_128 = np.zeros(896)
                        dimelo_gm_validA_single = np.nan                   
                gm_mA_ratio_128 = np.nan_to_num(dimelo_gm_modA_128/dimelo_gm_validA_128)
                gm_mA_ratio_single = np.nan_to_num(dimelo_gm_modA_single/dimelo_gm_validA_single) 
                # Scaling on these is 0 to 1, and we want 0 to 500
                f['128bp_bins'][index,:,13] = 500*gm_mA_ratio_128
                f['single_bin'][index,:,13] = 500*gm_mA_ratio_single
                
                with pyBigWig.open(f'{hek_dimelo_prefix}{chrom}/hek293_dimelo_feb2022_{chrom}_bigwig_modCpG.bw') as bw:
                    contigs = bw.chroms()
                    contig = list(contigs.keys())[0]
                    contig_length = contigs[contig]
                    if t2t_end<contig_length:
                        clean_values = np.nan_to_num(np.array(bw.values(contig,t2t_start,t2t_end)))
                        dimelo_hek_modC_128 = np.sum(clean_values.reshape(-1,128),axis=1)
                        dimelo_hek_modC_single = np.sum(clean_values)
                    else:
                        dimelo_hek_modC_128 = np.zeros(896)
                        dimelo_hek_modC_single = np.nan                       
                with pyBigWig.open(f'{hek_dimelo_prefix}{chrom}/hek293_dimelo_feb2022_{chrom}_bigwig_validCpG.bw') as bw:
                    contigs = bw.chroms()
                    contig = list(contigs.keys())[0]
                    contig_length = contigs[contig]
                    if t2t_end<contig_length:
                        clean_values = np.nan_to_num(np.array(bw.values(contig,t2t_start,t2t_end)))
                        dimelo_hek_validC_128 = np.sum(clean_values.reshape(-1,128),axis=1)
                        dimelo_hek_validC_single = np.sum(clean_values)  
                    else:
                        dimelo_hek_validC_128 = np.zeros(896)
                        dimelo_hek_validC_single = np.nan                                         
                hek_CpG_ratio_128 = np.nan_to_num(dimelo_hek_modC_128/dimelo_hek_validC_128)
                hek_CpG_ratio_single = np.nan_to_num(dimelo_hek_modC_single/dimelo_hek_validC_single) 
                # Scaling on these is 0 to 1, and we want 0 to 500
                f['128bp_bins'][index,:,14] = 500*hek_CpG_ratio_128
                f['single_bin'][index,:,14] = 500*hek_CpG_ratio_single
                # CpGs can happen a max of one in two bases, so a 10x and /100 scaling is roughly right
                # for 128bp and 114688bp bins
                f['128bp_bins'][index,:,15] = 10*dimelo_hek_validC_128
                f['single_bin'][index,:,15] = 0.01*dimelo_hek_validC_single                
                
                with pyBigWig.open(f'{gm_dimelo_prefix}{chrom}/gm12878_dimelo_feb2022_{chrom}_bigwig_modCpG.bw') as bw:
                    contigs = bw.chroms()
                    contig = list(contigs.keys())[0]
                    contig_length = contigs[contig]
                    if t2t_end<contig_length:
                        clean_values = np.nan_to_num(np.array(bw.values(contig,t2t_start,t2t_end)))
                        dimelo_gm_modC_128 = np.sum(clean_values.reshape(-1,128),axis=1)
                        dimelo_gm_modC_single = np.sum(clean_values)
                    else:
                        dimelo_gm_modC_128 = np.zeros(896)
                        dimelo_gm_modC_single = np.nan                      
                with pyBigWig.open(f'{gm_dimelo_prefix}{chrom}/gm12878_dimelo_feb2022_{chrom}_bigwig_validCpG.bw') as bw:
                    contigs = bw.chroms()
                    contig = list(contigs.keys())[0]
                    contig_length = contigs[contig]
                    if t2t_end<contig_length:
                        clean_values = np.nan_to_num(np.array(bw.values(contig,t2t_start,t2t_end)))
                        dimelo_gm_validC_128 = np.sum(clean_values.reshape(-1,128),axis=1)
                        dimelo_gm_validC_single = np.sum(clean_values)
                    else:
                        dimelo_gm_validC_128 = np.zeros(896)
                        dimelo_gm_validC_single = np.nan                  
                gm_CpG_ratio_128 = np.nan_to_num(dimelo_gm_modC_128/dimelo_gm_validC_128)
                gm_CpG_ratio_single = np.nan_to_num(dimelo_gm_modC_single/dimelo_gm_validC_single)
                f['128bp_bins'][index,:,16] = 500*gm_CpG_ratio_128
                f['single_bin'][index,:,16] = 500*gm_CpG_ratio_single
                # CpGs can happen a max of one in two bases, so a 10x and /100 scaling is roughly right
                # for 128bp and 114688bp bins
                f['128bp_bins'][index,:,17] = 10*dimelo_gm_validC_128
                f['single_bin'][index,:,17] = 0.01*dimelo_gm_validC_single  
                
np.seterr(**old_settings)

100%|██████████| 38171/38171 [4:33:20<00:00,  2.33it/s]   


{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}